In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import unicodedata

In [2]:
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html'

### Extract date

In [3]:
content = requests.get(url).content
soup =BeautifulSoup(content, 'lxml')

In [4]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

data = soup.findAll(text=re.compile(".*pdated .*202"))
data


['Last updated on April 30, 2020']

In [5]:
if len(data)>0:
    m = re.search(".*pdated .* (\w+ \d+, 202\d)", _removeNonAscii(data[0]))
    data_date_str = m.group(1)
    data_date = datetime.strptime(data_date_str, '%B %d, %Y').strftime('%Y-%m-%d')
else:
    data_date = datetime.utcnow().strftime('%Y-%m-%d')
data_date

'2020-04-30'

### Extract data and add into Dataframe

In [6]:
dfs = pd.read_html(url)

In [7]:
if len(dfs) == 0 :
    raise Exception('There is no dataframes detected')

In [8]:
dfs

[                           Unnamed: 0_level_0  \
                             Age group (years)   
 0                                      Totals   
 1                    Race missing/unspecified   
 2                              Race specified   
 3             Among those with race specified   
 4            American Indian or Alaska Native   
 5                                       Asian   
 6                   Black or African American   
 7   Native Hawaiian or other Pacific Islander   
 8                                       White   
 9                              Multiple/other   
 10                                        NaN   
 11              Ethnicity missing/unspecified   
 12                        Ethnicity specified   
 13       Among those with ethnicity specified   
 14                            Hispanic/Latino   
 15                        Non-Hispanic/Latino   
 
                No. of cases (% of total)  \
                                     < 18   
 0      

In [9]:
df = dfs[0]
df

Unnamed: 0_level_0  \
                            Age group (years)   
0                                      Totals   
1                    Race missing/unspecified   
2                              Race specified   
3             Among those with race specified   
4            American Indian or Alaska Native   
5                                       Asian   
6                   Black or African American   
7   Native Hawaiian or other Pacific Islander   
8                                       White   
9                              Multiple/other   
10                                        NaN   
11              Ethnicity missing/unspecified   
12                        Ethnicity specified   
13       Among those with ethnicity specified   
14                            Hispanic/Latino   
15                        Non-Hispanic/Latino   

               No. of cases (% of total)  \
                                    < 18   
0                                  16980   
1                          11,293(66.5%)   
2                           5,687(33.5%)   
3        Among those with race specified   
4                               95(1.7%)   
5                              268(4.7%)   
6                           1,339(23.5%)   
7                               37(0.7%)   
8                           2,851(50.1%)   
9                           1,097(19.3%)   
10                                   NaN   
11                         11,215(66.0%)   
12                          5,765(34.0%)   
13  Among those with ethnicity specified   
14                          2,280(39.5%)   
15                          3,485(60.5%)   

                                          \
                                   18-44   
0                                 286823   
1                         177,797(62.0%)   
2                         109,026(38.0%)   
3        Among those with race specified   
4                              992(0.9%)   
5                            5,849(5.4%)   
6                          30,217(27.7%)   
7                              433(0.4%)   
8                          54,101(49.6%)   
9                          17,434(16.0%)   
10                                   NaN   
11                        185,756(64.8%)   
12                        101,067(35.2%)   
13  Among those with ethnicity specified   
14                         31,677(31.3%)   
15                         69,390(68.7%)   

                                          \
                                   45-64   
0                                 283597   
1                         163,748(57.7%)   
2                         119,849(42.3%)   
3        Among those with race specified   
4                              859(0.7%)   
5                            6,395(5.3%)   
6                          37,585(31.4%)   
7                              343(0.3%)   
8                          58,531(48.8%)   
9                          16,136(13.5%)   
10                                   NaN   
11                        175,877(62.0%)   
12                        107,720(38.0%)   
13  Among those with ethnicity specified   
14                         27,160(25.2%)   
15                         80,560(74.8%)   

                                          \
                                   65-74   
0                                  88249   
1                          42,289(47.9%)   
2                          45,960(52.1%)   
3        Among those with race specified   
4                              240(0.5%)   
5                            2,141(4.7%)   
6                          14,771(32.1%)   
7                              103(0.2%)   
8                          24,182(52.6%)   
9                            4,523(9.8%)   
10                                   NaN   
11                         48,764(55.3%)   
12                         39,485(44.7%)   
13  Among those with ethnicity specified   
14                          6,988(17.7%)   
15               

In [10]:
csv_file='../data/cdc-us-cases-by-races-and-age-group.csv'
df.to_csv(csv_file, index=False)

In [11]:
# col0 = df.columns[0]
# df = df.set_index(col0).T
# df



In [12]:
# # remove non alphanumeric characters from the columns
# # also lowercase them
# df.columns = df.columns.map(lambda x: re.sub('\W+', '', str.lower(x)))

# # add column date to the front
# df.insert(0, 'date', data_date)


### Save to file (old data)


In [13]:
# from pathlib import Path
# csv_file = '../data/cdc-us-cases.csv'

# if Path(csv_file).exists():
#     # read out the old dat
#     old_df = pd.read_csv(csv_file)
#     df = df.append(old_df)
# df.drop_duplicates(subset=['date'], inplace=True)

In [14]:
# df.sort_values(by=['date'], inplace=True)
# df
# df.to_csv(csv_file, index=False)